In [2]:
import pandas as pd
import requests
import string

In [3]:
# Import data
data_init = pd.read_csv('data/nlp-data.csv')

In [4]:
# Dataframe containg the data that our team created
data_init.head()

,Text Command,Topic,Identifier,Action,One Hot Encoded Topic,One Hot Encoded Action
0,Assign completed status to X,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
1,Move X to completed,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
2,X completed,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
3,Completed X,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
4,Started on Xverb,Task,Xverb,In Progress,"[1,0,0]","[0,1,0,0,0]"


In [5]:
# Script for creating new data by substituting words for their synonyms. Only the words which are not included
# in the identifier are eligible for being replaced
def aug_data(df, syn_num):
    #new dataframe for the new sentences
    added_data = pd.DataFrame(columns=list(df.columns))
    index = len(df)
    for i in range(len(df.index)):
        identifier = df.iloc[i][2]
        text = df.iloc[i][0]
        sentence_lst = text.split()
        # identifier_lst = identifier.split()
        
        # List of additional words that translating did not make sense
        identifier_lst = ['a', 'an']
        identifier_lst.append(identifier.split()[0])

        for j in range(len(sentence_lst)):
            word = sentence_lst[j]
            if word not in identifier_lst:
                word = word.lower()
                
                # max=syn_num is the number of synonyms you want for each non-identifier word in the cmd
                
                api_url = 'https://api.datamuse.com/words?rel_syn={0}&max='.format(word)
                api_url += str(syn_num)
                word_synonyms = requests.get(api_url)
                word_synonyms = word_synonyms.json()
                
                for k in range(len(word_synonyms)):
                    new_sentence_lst = sentence_lst[:j]+[word_synonyms[k]['word']]+sentence_lst[j+1:]
                    s = " "
                    new_sentence = s.join(new_sentence_lst)
                    added_data.loc[index] = [new_sentence] + list(df.loc[i][1:])
                    index += 1
                    
    augmented_data = pd.concat([df,added_data])
    return augmented_data

Augmenting specified actions/rows for exploration purposes:

In [ ]:
# pd.set_option("display.max_rows", None)

In [6]:
# Limiting data to just completed tasks for API exploration purposes
data_action5 = data_init[data_init.Action == 'Completed'].reset_index(drop=True)
data_action5

,Text Command,Topic,Identifier,Action,One Hot Encoded Topic,One Hot Encoded Action
0,Assign completed status to X,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
1,Move X to completed,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
2,X completed,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
3,Completed X,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
4,X is completed,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
5,X is now completed,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
6,Finally completed X,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
7,Completed Xtime earlier,Task,Xtime,Completed,"[1,0,0]","[0,0,0,0,1]"
8,Completed Xtime yesterday,Task,Xtime,Completed,"[1,0,0]","[0,0,0,0,1]"
9,Finished with Y,Task,Y,Completed,"[1,0,0]","[0,0,0,0,1]"


In [9]:
# Running the script for just the first X rows
data_action5_aug = aug_data(data_action5.iloc[0:5], 5)

In [10]:
data_action5_aug

,Text Command,Topic,Identifier,Action,One Hot Encoded Topic,One Hot Encoded Action
0,Assign completed status to X,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
1,Move X to completed,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
2,X completed,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
3,Completed X,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
4,X is completed,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
5,put completed status to X,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
6,attribute completed status to X,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
7,ascribe completed status to X,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
8,delegate completed status to X,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"
9,designate completed status to X,Task,X,Completed,"[1,0,0]","[0,0,0,0,1]"


Augmenting current data:

In [12]:
new_data = aug_data(data_init, 5)

In [32]:
new_data.loc[500:600]

,Text Command,Topic,Identifier,Action,One Hot Encoded Topic,One Hot Encoded Action
500,Task X is in critique,Task,X,In Review,"[1,0,0]","[0,0,1,0,0]"
501,Task X is in recap,Task,X,In Review,"[1,0,0]","[0,0,1,0,0]"
502,Task X is in refresh,Task,X,In Review,"[1,0,0]","[0,0,1,0,0]"
503,X job is in review,Task,X,In Review,"[1,0,0]","[0,0,1,0,0]"
504,X project is in review,Task,X,In Review,"[1,0,0]","[0,0,1,0,0]"
...,...,...,...,...,...,...
596,Review X labor,Task,X,In Review,"[1,0,0]","[0,0,1,0,0]"
597,Review X chore,Task,X,In Review,"[1,0,0]","[0,0,1,0,0]"
598,X survey,Task,X,In Review,"[1,0,0]","[0,0,1,0,0]"
599,X follow-up,Task,X,In Review,"[1,0,0]","[0,0,1,0,0]"


Hello Word
